# `InfiniteOpt.jl`: The Basics
Now that we are familiarized with Julia and `JuMP.jl`, let's talk about how `InfiniteOpt.jl` builds upon `JuMP.jl` to allow us to intuitively model infinite-dimensional optimization problems.

## Resources
Again, there is only so much that we can cover in a tutorial. Helpful resources include:
- The documentation: https://pulsipher.github.io/InfiniteOpt.jl/stable/.
- The paper: https://www.sciencedirect.com/science/article/pii/S0098135421003458?via%3Dihub 
- The open-source paper: https://arxiv.org/abs/2106.12689
- The `InfiniteOpt.jl` forum: https://github.com/pulsipher/InfiniteOpt.jl/discussions

## Optimal Control in `JuMP.jl`
To motivate `InfiniteOpt.jl`, let's consider modeling a simple optimal control problem in `JuMP.jl`.

### The Formulation
We'll take a look at a pandemic control problem that uses the SEIR model (a system of ODEs):
$$
\begin{aligned}
&&\min_{} &&& \int_{t \in \mathcal{D}_{t}} u(t) dt \\
&& \text{s.t.} &&& \frac{d s(t)}{d t} = (u(t) - 1)\beta s(t)i(t), && t \in \mathcal{D}_{t}\\
&&&&& \frac{d e(t)}{d t} = (1 - u(t))\beta s(t)i(t) - \xi e(t), && t \in \mathcal{D}_{t} \\
&&&&& \frac{d i(t)}{d t} = \xi e(t) - \gamma i(t), && t \in \mathcal{D}_{t}\\
&&&&& \frac{d r(t)}{d t} = \gamma i(t), && t \in \mathcal{D}_{t} \\
&&&&& i(t) \leq i_{max}, && t \in \mathcal{D}_{t} \\
&&&&& u(t) \in [0, 0.8], && t \in \mathcal{D}_{t} \\
&&&&& s(0) = s_0, e(0) = e_0, i(0) = i_0, r(0) = r_0 \\
\end{aligned}
$$
Here $s(t)$, $e(t)$, $i(t)$, and $r(t)$ are state variables. The control variable $u(t)$ controls the mandated isolation policy on the population. The remaining quantities are parameters of the contagion.

We cannot directly model this with `JuMP.jl` because the variables are functions of time $t$ and the formulation contains an integrals/derivatives. This is an infinite-dimensional problem.

### Discretize
We can transform it into a finite optimization `JuMP.jl` can handle by discretizing the variables over time and applying appropriate approximations to the integral/derivatives. 

We will transform the model via:
- let $\mathcal{D}_t = [0, T]$ and choose $n$ time steps such that $\Delta t = \frac{T}{n}$
- discretize each infinite variable (e.g., $\{s_t: t \in \{0, 1, \dots, n\}$)
- replace the integral with a sum over time
- approximate the derivatives via explicit Euler

With these steps, we obtain:
$$
\begin{aligned}
&&\min_{} &&& \sum_{t \in \{1, \dots, n\}} u_t \\
&& \text{s.t.} &&& s_{t+1} = s_t + \Delta t(u_t - 1)\beta s_ti_t, && t \in \{0, 1, \dots, n-1\}\\
&&&&& e_{t+1} = e_t + \Delta t((1 - u_t)\beta s_ti_t - \xi e_t), && t \in \{0, 1, \dots, n-1\} \\
&&&&& i_{t+1} = i_t + \Delta t(\xi e_t - \gamma i_t), && t \in \{0, 1, \dots, n-1\}\\
&&&&& r_{t+1} = r_t + \Delta t \gamma i_t, && t \in \{0, 1, \dots, n-1\} \\
&&&&& i_t \leq i_{max}, && t \in \{1, \dots, n\} \\
&&&&& u_t \in [0, 0.8], && t \in \{1, \dots, n\} \\
&&&&& s_0 = s_0, e_0 = e_0, i_0 = i_0, r_0 = r_0 \\
\end{aligned}
$$

This is a discrete time model that we can formulate with `JuMP.jl`, let's try it out!

### Exercise: Optimal Control in `JuMP.jl`
**Problem**
- Implement the above model in `JuMP.jl`

In [1]:
using JuMP, Ipopt, Plots

# Set the parameters
γ = 0.303
β = 0.727
N = 1e5
ξ = 0.3
i_max = 0.02
T = 200
n = 200
Δt = T / n

# Set the intial condition values
e0 = 1 / N
i0 = 0
r0 = 0
s0 = 1 - 1 / N

# Create the JuMP.jl Model
model = Model(Ipopt.Optimizer)

@variable(model, s[0:n])
@variable(model, e[0:n])
@variable(model, i[0:n])
@variable(model, r[0:n])
@variable(model, 0 <= u[0:n] <= 0.8)

@objective(model, Min, sum(u))

@NLconstraint(model, [t ∈ 0:n-1], s[t+1] == s[t] + Δt * (u[t+1] - 1) * β * s[t+1] * i[t+1])
@NLconstraint(model, [t ∈ 0:n-1], e[t+1] == e[t] + Δt * ((1 - u[t+1]) * β * s[t+1] * i[t+1] - ξ * e[t+1]))
@constraint(model, [t ∈ 0:n-1], i[t+1] == i[t] + Δt * (ξ * e[t+1] - γ * i[t+1]))
@constraint(model, [t ∈ 0:n-1], r[t+1] == r[t] + Δt * γ * i[t+1])
@constraint(model, [t ∈ 1:n], i[t] <= i_max)

fix(s[0], s0)
fix(e[0], e0)
fix(i[0], i0)
fix(r[0], r0)
fix(u[0], 0, force = true)

optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     2996
Number of nonzeros in inequality constraint Jacobian.:      200
Number of nonzeros in Lagrangian Hessian.............:     2400

Total number of variables............................:     1000
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      200
                     variables with only upper bounds:        0
Total number of equality constraints.................:      800
Total number of inequality co